In [ ]:
import typing
import colorgram
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

우선 파일을 불러오자.

In [ ]:
image = cv2.imread('../images/IMG_9653.png')

영상처리를 하기에 앞서, 여러 이미지에 대해 비슷한 결과를 출력하길 바라는 마음에 이미지의 일부를 추출하여 업샘플링/다운샘플링하여 사용할 것이다.

너비와 높이 중 짧은 것의 80%를 한 변의 길이로 하는 정사각형을 추출하여 사용하겠다.

그리고, 항상 고정된 해상도 512x512를 기준으로 영상을 처리하겠다.

In [ ]:
center_y, center_x = image.shape[0]/2, image.shape[1]/2
radius = 0.4 * min(image.shape[:2])

y_min = int(center_y-radius)
y_max = int(center_y+radius)
x_min = int(center_x-radius)
x_max = int(center_x+radius)

image = image[y_min:y_max, x_min:x_max, :]
image = cv2.resize(image, (512, 512))

# 영상 분석

이제, 어떻게 하면 위 이미지로부터 알약을 찾을 수 있을 지 알아보자.

입력된 이미지에서 각 픽셀의 값들이 RGB 공간에서 어떻게 분포되어있는지를 보고자 한다.

In [ ]:
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
r, g, b = cv2.split(rgb_image)

fig = plt.figure()

ax1 = fig.add_subplot(1, 3, 1)
ax1.imshow(rgb_image)

ax2 = fig.add_subplot(1, 3, 2, projection='3d')
ax2.scatter(r, g, b, marker='.')
ax2.set_xlabel('R')
ax2.set_ylabel('G')
ax2.set_zlabel('B')
ax2.view_init(45, -135)

ax3 = fig.add_subplot(1, 3, 3, projection='3d')
ax3.scatter(r, g, b, marker='.')
ax3.set_xlabel('R')
ax3.set_ylabel('G')
ax3.set_zlabel('B')
ax3.view_init(30, -45)

x, y, z 축이 균일하게 상승하는 중앙 곡선은 아마도 무채색의 배경일 것이다.

그리고, 한 쪽으로 튀어나온 부분이 파란색의 알약이 있는 부분 일 것이다.

잠깐 정리하고 넘어가보자면, 이미지로 부터 알약을 찾기 위해 필요한 정보는 다음과 같다.

1. 알약의 색상
2. 알약의 모양
3. (...)

# 색상 추출

알약의 색상을 찾기위해서는 우선 채도를 바탕으로 색이 있는지 없는지를 판단할 필요가 있다. 색상이 있다고 판단되면 그 때 색상을 찾는다.

색상이 있고 없음을 구분지을 수 있는 속성은 아무래도 채도일 것이다. 
특히 조명에 의한 밝기 변화 대하여 채도는 명도에 비해 영향을 덜 받기 때문에 조명에도 강할 것이다.

관련 색 공간으로 기저를 변환해 보는 것이 좋을 것 같다.

RGB -> HSV로 변환해보자.

In [ ]:
hsv_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)
h, s, v = cv2.split(hsv_image)

각 픽셀의 채도의 출력하여, 채도가 알약 검출에 사용될 수 있을지를 가늠해보자.

In [ ]:
gray_image = s

plt.imshow(gray_image, cmap='gray')

채도에 대한 채널만 출력했음에도 마스크 이미지에 가까운 결과를 얻을 수 있었다.

흑백이미지를 이진(마스크)이미지로 만들기 위해 임계값처리를 해주고,
노이즈를 제거하기 위해 모폴로지 연산을 수행해볼 것이다.

먼저 임계값 처리를 하기 위해, 히스토그램 분석을 통해 적절한 임계값을 찾는다.

In [ ]:
plt.hist(gray_image)

혹시 히스토그램 평활화를 쓴다면, 개선이 될까?

오히려 경계가 더 허물어져서 안한 것 만 못할 것 같지만, 호기심에 한 번 시도해보았다.

In [ ]:
gray_image_equalized = cv2.equalizeHist(gray_image)

plt.subplot(121)
plt.hist(gray_image_equalized)
plt.subplot(122)
plt.imshow(gray_image_equalized, cmap='gray')

낮은 값에 배경이 몰려있고, 약 [160, 220] 값 사이에 알약이 위치했음으로 추정해 볼 수 있을 것 같다.

우리는 눈으로 보고 적절히 150에서 임계값을 잡으면 될 것이라고 생각해 볼 수 있겠으나, 다양한 이미지로부터 알약을 찾도록 확장한다고 생각해보면 매번 사용자가 직접 임계값을 설정하는 것은 굉장히 비효율적이며 부적절할 것이다.

임계값을 자동으로 찾기위한 여러 알고리즘 중, Otsu의 이진화 알고리즘을 사용해보려고 한다.

Otsu 알고리즘은 두 클래스의 분산을 최소화시키는 임계값을 찾기위한 알고리즘이다.

<figure>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/Otsu%27s_Method_Visualization.gif/440px-Otsu%27s_Method_Visualization.gif"/>
    <figcaption>Otsu's method visualization</figcaption>
</figure>

https://en.wikipedia.org/wiki/Otsu%27s_method

In [ ]:
otsu_value, bin_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print(f"Otsu's threshold value: {otsu_value}")
plt.imshow(bin_image, cmap='gray')

Otsu의 이진화 알고리즘은 97을 임계값으로 설정한 것 같다.
나쁘지 않아보이므로, 이를 그대로 사용해보겠다.

이어서, 모폴로지 침식-팽창 시퀀스로 점노이즈를 제거해주었고, 팽창-침식 시퀀스로 홀을 제거해주었다.

In [ ]:
kernel = np.ones((9, 9))
bin_image = cv2.morphologyEx(bin_image, cv2.MORPH_OPEN, kernel)
bin_image = cv2.morphologyEx(bin_image, cv2.MORPH_CLOSE, kernel)

plt.imshow(bin_image, cmap='gray')

# 색상이 없다면?

만약에 색상이 없는 알약은 어떻게 검출해야 할까.
이 경우에는 알약의 에지를 검출하거나, 그 외에 다른 방법을 사용해야 한다.
에지를 검출할 경우 그림자가 에지성분에 포함될 수 있는 문제가 있다.

향후 모양 판별을 함에 있어 마스크 이미지의 정확도가 상당히 중요한 요인이기 때문에, 에지 검출을 이용한다면 꽤 많은 어려움을 겪게 된다.

따라서, 본 실습에서는 그레이 스케일의 알약에 대해서는 배경을 알약과 확연히 다른 색상으로 선택하여 알약을 구분하는 것으로 대채한다. 물론 이 경우에는 HSV대신 다른 색공간을 이용하는 것 또한 고려해봐야 한다.
(예: 하얀 알약에 대해서는 검정색 배경 사용 -> 밝기 값 사용)

# 그래서 이 색상은 무엇인가

알약의 위치를 특정했으니, 이 알약의 색이 무엇인지를 알아낼 수 있어야 한다.

이는 다음과 같이 특정할 것이다.

1. 알약의 원본 이미지로 부터 마스크 이미지를 사용해 배경을 제거한다.
2. 배경이 제거된 영상에서 픽셀별 색상 분포에 대한 통계를 낸다.
3. 가장 많은 색상은 배경색상이므로, 두 번째로 가장 많은 색상이 알약의 색상이 된다.

2번에서 통계를 내는 과정은 외부 라이브러리(colorgram.py)를 사용한다.

In [ ]:
# 마스킹을 통해 알약의 영역만 추출
pill_image = cv2.copyTo(rgb_image, bin_image)

# 다운샘플링으로 처리속도 개선
# (colorgram.extract() 는 작업 시간이 크다. 실행시간 단축을 위해 다운샘플링을 사용한다.)
down_sampled_image = cv2.resize(pill_image, (64, 64))

# 색상 통계 및 분석
colors: typing.List[colorgram.Color]

pillow_image = Image.fromarray(down_sampled_image)
colors = colorgram.extract(pillow_image, number_of_colors=2)

# 두 번째로 많은 색상 추출
pill_color = colors[1].rgb

# 모양 판별

이제 모양을 판별할 차례이다.

모양을 판별하는 방법으로는 Correlation 패턴인식 혹은 윤곽선 검출 응용등의 방법이 있다.

처음에는 이 중에서 Correlation 패턴인식을 사용해 볼 생각이었으나, 곧 해당 방법의 효용성에 의문이 들기 시작했다.
알약의 모양은 크게 원형, 장방형, 원통형, 삼각형 등 유사한 범주로 구분이 가능하지만, 알약마다 변의 길이가 제각각이기에 Correlation 만으로는 이들을 구분하기 어려울 것 같다는 생각이 들었다.

따라서, 윤곽선을 이용한 검출 방법을 택하기로 하였다.

먼저 위 이미지에서 윤곽선을 검출한다.

In [ ]:
contours = cv2.findContours(bin_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2]

이제 검출한 윤곽선으로 부터 모양을 추출할 것이다.

모양을 추출하는 방법으로는 꼭짓점의 개수를 헤아리는 것으로 근사하게 추출 할 수 있다.

가령 꼭짓점이 3개가 검출되었다면 이는 삼각형임을 의미하고, 꼭짓점이 없다(무수히 많다)의 경우 이는 원이라고 판단할 수 있다.

하지만 2차원 이미지에서 얻어진 윤곽선은 울퉁불퉁한 픽셀들로부터 추출한 윤곽선이므로 원치않는 꼭짓점 노이즈가 검출되는 일이 잦을 수 밖에 없다. 따라서, 윤곽선을 근사시키는 방법을 활용한다. 근사의 기준이 되는 근사치 정확도는 둘레의 총 길이의 일정 비율(4%)로 설정하여, 서로다른 크기의 유사한 모양에 대해서도 효과적이도록 설계할 것이다.

여기서 어려운 부분은 원형과 타원형, 그리고 장방형의 구분이다. 

아래 코드에서는 구분하기 쉬운 삼각형, 사각형 그리고 오각형을 우선적으로 구분한다.

In [ ]:
print(f'{len(contours)} contours found.')

if len(contours) > 1:
    raise Exception('아직 동시에 여러 개의 알약은 검출하기 어렵습니다.')

contour = contours[0]
approx = cv2.approxPolyDP(contour, 0.04 * cv2.arcLength(contour, True), True)
n_approx = len(approx) # 꼭짓점의 개수

if n_approx == 3:
    print('triangle')
elif n_approx == 4:
    print('quadrilateral')
elif n_approx == 5:
    print('pentagon')
else:
    print('circle? oval? rectangular?')
    print(n_approx)

찾아진 윤곽선과 꼭짓점들을 표시하면 다음과 같이 나타낼 수 있다.

In [ ]:
temp_image = np.zeros((512, 512, 3), dtype=np.uint8)
temp_image[bin_image == 255,:] = 255
cv2.drawContours(temp_image, [contour], 0, (255, 0, 0), 2)

x = approx[:,:,0]
y = approx[:,:,1]

plt.subplot(121)
plt.imshow(bin_image, cmap='gray')

plt.subplot(122)
plt.imshow(temp_image, cmap='gray')
plt.scatter(x, y, marker='o')

# 모양판별 Tie break

(circle, oval, rectangular)

이제 구분이 어려운 모양들을 구분 할 것이다.

<figure>
    <img src="../images/IMG_9801.png" height="320"/>
    <figcaption>패턴인식으로 구별이 어려운 장방형 알약들</figcaption>
</figure>

In [ ]:
def create_pill_mask(image, is_colored=True):
    pass